In [2]:
from langchain_ollama.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.prompts import PromptTemplate
from pprint import pprint
from langchain_core.pydantic_v1 import BaseModel, Field

In [3]:

# llm =  ChatOllama(model="phi3:3.8b")
llm =  ChatOllama(model="cow/gemma2_tools:2b")

In [4]:
from langchain.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
loader = WebBaseLoader("https://en.wikipedia.org/wiki/Munich")

In [6]:
raw_docs = loader.load()

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings


In [9]:
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/tmp/ipykernel_13139/3414252995.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [11]:
splitter = RecursiveCharacterTextSplitter(chunk_size=750, chunk_overlap=150, length_function=len,separators=["\n\n","\n"])

In [12]:
DOCS = splitter.split_documents(raw_docs)

In [13]:
len(DOCS)

295

In [14]:
DB = FAISS.from_documents(DOCS, embed_model)

In [16]:
DB.similarity_search("Do people use bicycles in Munich?",k=3)

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Munich', 'title': 'Munich - Wikipedia', 'language': 'en'}, page_content="Cycling[edit]\nMain article: Cycling in Munich\nMap of Munich's cycling network\nCycling has a strong presence in the city and is recognized as a good alternative. The growing number of bicycle lanes are widely used throughout the year. Cycle paths can be found alongside the majority of sidewalks and streets, although the newer or renovated ones are much easier to tell apart from pavements than older ones. A modern bike hire system is available within the area bounded by the Mittlerer Ring."),
 Document(metadata={'source': 'https://en.wikipedia.org/wiki/Munich', 'title': 'Munich - Wikipedia', 'language': 'en'}, page_content='Cultural history trails and bicycle routes[edit]\nSince 2001, historically interesting places in Munich can be explored via the List of cultural history trails in Munich (KulturGeschichtsPfade). Sign-posted cycle routes are the Outer

In [31]:
template = """
You are a helful bot. Your task is to find the answer from the given context.
If answer not in the context just say I dont know. 
Dont assume answers if not int context.

Context : {context}
Question: {question}
"""

prompt = PromptTemplate.from_template(template)

In [32]:
def format_docs(question):
    docs = [i.page_content for i in DB.similarity_search(question)]
    docs = "\n\n".join(docs)
    return docs

In [33]:
from langchain_core.runnables import RunnablePassthrough
chain = ({"context": format_docs, "question": RunnablePassthrough() } | prompt | llm |  StrOutputParser())

In [34]:
chain.invoke("Do people use bicycles in Munich?")

'Yes. Cycling is a strong presence in the city and recognized as a good alternative. The growing number of bicycle lanes are widely used throughout the year.\r\n\r\n'

In [35]:
chain.invoke("Which city has strongest economy of any German city ")

'Munich.\r\n'

In [36]:
chain.invoke("Which city has lowest un employment rate in german")

'The answer to your question can be found in the text provided: "Munich has the lowest unemployment rate (5.4% in July 2020) of any German city of more than a million people (the others being Berlin, Hamburg and Cologne)." \r\n\r\n**Therefore, Munich has the lowest unemployment rate in Germany.** \r\n'